In [1]:
import os, ROOT

os.environ["ROOT_INCLUDE_PATH"] = os.path.join(os.environ["CONDA_PREFIX"], "include")
os.environ["LD_LIBRARY_PATH"] = os.path.join(os.environ["CONDA_PREFIX"], "lib") + ":" + os.environ.get("LD_LIBRARY_PATH", "")

ROOT.gErrorIgnoreLevel = ROOT.kError  # hide info/warning spam
ROOT.EnableImplicitMT()

In [2]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
# --- ATLAS style ---
ROOT.gROOT.LoadMacro("/home/kalelc/research/atlasstyle/AtlasStyle.C")
ROOT.gROOT.LoadMacro("/home/kalelc/research/atlasstyle/AtlasUtils.C")
ROOT.gROOT.LoadMacro("/home/kalelc/research/atlasstyle/AtlasLabels.C")
ROOT.SetAtlasStyle()

ROOT.EnableImplicitMT()

# --- Load Delphes ---
delphes_dir = "/home/kalelc/research/Delphes-3.5.0/"
status = ROOT.gSystem.Load(delphes_dir + "libDelphes.so")
if status != 0:
    raise RuntimeError("Error: Could not load libDelphes.so")
print("libDelphes.so loaded successfully!")

libDelphes.so loaded successfully!

Applying ATLAS style settings...



Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libROOTNTuple_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libMatrix_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libHist_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libGraf_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libGpad_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libGraf3d_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libEG_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libTreePlayer_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/miniforge3/lib/libROOTVecOps_rdict.pcm file does not exist
Error in <TCling::LoadPCM>: ROOT PCM /home/kalelc/min

In [4]:
# Input ROOT file
fdir = delphes_dir + "displacedPhoton/prompt_half_dark_events.root"
df = ROOT.RDataFrame("Delphes", fdir)

PHI_FINAL_STATUS = 62
FINAL_STATUS = 1
DPHOTON_PID = 4900022
PHOTON_PID = 22
PHI_PID = 54

EVENT_FILTER = 15

In [5]:
# --- Convert truth particles to pandas ---
particle_df = ROOT.RDataFrame("Delphes", fdir)
particle_dict = particle_df.AsNumpy([
    "Event.Number",
    "Particle.PT",
    "Particle.Phi",
    "Particle.Eta",
    "Particle.T",
    "Particle.PID",
    "Particle.fUniqueID",
    "Particle.M1",
    "Particle.M2",
    "Particle.D1",
    "Particle.D2",
    "Particle.Status",
])

# Flatten event numbers (scalars, not 1-element arrays)
event_numbers = [arr[0] for arr in particle_dict["Event.Number"]]
# df = df.Filter("Event.Number < {}".format(EVENT_FILTER), "Events below event filter")

# Add particle indices per event (needed for mother/daughter lookup)
particle_dict["Particle.Index"] = [
    np.arange(len(pt)) for pt in particle_dict["Particle.PT"]
]

# Build DataFrame
particle_df = pd.DataFrame({
    "event": event_numbers,
    "ParticleIndex": particle_dict["Particle.Index"],
    "PT": particle_dict["Particle.PT"],
    "Phi": particle_dict["Particle.Phi"],
    "Eta": particle_dict["Particle.Eta"],
    "T": particle_dict["Particle.T"],
    "PID": particle_dict["Particle.PID"],
    "UniqueID": particle_dict["Particle.fUniqueID"],  # keep for reference
    "M1": particle_dict["Particle.M1"],
    "M2": particle_dict["Particle.M2"],
    "D1": particle_dict["Particle.D1"],
    "D2": particle_dict["Particle.D2"],
    "Status": particle_dict["Particle.Status"],
})

# Event Filter
particle_df = particle_df[(particle_df["event"] < EVENT_FILTER)]

# Explode so each particle gets its own row
particle_df = particle_df.explode([
    "ParticleIndex", "PT", "Phi", "Eta", "T", "PID", "UniqueID",
    "M1", "M2", "D1", "D2", "Status"
], ignore_index=True)

# Convert indices to integer
particle_df[["ParticleIndex", "M1", "M2", "D1", "D2"]] = particle_df[["ParticleIndex", "M1", "M2", "D1", "D2"]].astype(int)

particle_df = particle_df.sort_values(by="event").reset_index(drop=True)

particle_df.head(5)

,event,ParticleIndex,PT,Phi,Eta,T,PID,UniqueID,M1,M2,D1,D2,Status
0,0,0,0.0,0.0,999.900024,0.0,2212,1,-1,-1,2,9,4
1,0,1,0.0,0.0,-999.900024,0.0,2212,10,-1,-1,10,17,4
2,0,2,3.147123,0.045965,5.993112,0.0,21,2,0,-1,18,18,61
3,0,3,2.080813,-1.431776,6.782842,0.0,2,3,0,-1,19,19,61
4,0,4,2.631126,2.575235,0.541287,0.0,21,4,0,-1,20,20,61


In [6]:
def track_parents(df, unique_id, event_number, track):
    """Track the single parentage of a particle given its unique ID and event number through M1.

    Args:
        df (pd.DataFrame): DataFrame containing particle information.
        unique_id (int): Unique ID of the particle to track.
        event_number (int): Event number of the particle.
        track (list): List to store the track of parent particles.
    """
    # Get the current particle's information
    current_particle = df[(df["event"] == event_number) & (df["UniqueID"] == unique_id)]
    if current_particle.empty:
        return

    # Get the mother particle's index
    mother_index = current_particle["M1"].values[0]
    if mother_index == -1:
        return

    # Append the current particle's unique ID to the track
    track.append(unique_id)

    # Recursively track the mother particle
    track_parents(df, mother_index, event_number, track)
    return track

def filter_photon_ids(df, event_number, state=1):
    """Get the unique IDs of all photons in a given event.

    Args:
        df (pd.DataFrame): DataFrame containing particle information.
        event_number (int): Event number to search for photons.
    """
    df = df[(df["event"] == event_number) & (df["PID"] == 22) & (df["Status"] == state)]
    df = df[(df["PT"] > 10) & (df["Eta"].abs() < 2.5)]
    return df["UniqueID"].values

In [12]:
l_photon = filter_photon_ids(particle_df, 12)
for i, uid in enumerate(l_photon):
    track = []
    track_parents(particle_df, uid, 12, track)
    print("Photon UniqueID:", uid)
    print("Parentage Track (UniqueIDs):", track)
    if i >= 10:
        break

Photon UniqueID: 1279
Parentage Track (UniqueIDs): [1279, np.int64(1276), np.int64(1253), np.int64(1206), np.int64(1133), np.int64(1002), np.int64(737), np.int64(33), np.int64(16)]
Photon UniqueID: 1280
Parentage Track (UniqueIDs): [1280, np.int64(1276), np.int64(1253), np.int64(1206), np.int64(1133), np.int64(1002), np.int64(737), np.int64(33), np.int64(16)]
